TODO Придумать как учитывать Kernel.calc_grad_mean для решения о делении ядра
TODO Перенести код в библиотеку fractalnn.py
TODO Согласовать код с библиотекой generate-nn/nbmodule.py
TODO Заменить в Kernel.left, Kernel.right на Kernel.childs. Поменять соответствующие функции.
TODO Упростить инициализацию Kernel: первичная - без параметров
TODO Добавить в Kernel параметр kernel_size
TODO Возможно хватит квадратной матрицы Kernel?


In [1]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from enum import Enum
import copy
import random


In [2]:
l = nn.Linear(2, 4)
print(l.weight)
print(l.bias)

Parameter containing:
tensor([[-0.4147, -0.0406],
        [-0.4577,  0.0653],
        [-0.1847, -0.2568],
        [ 0.6002, -0.2063]], requires_grad=True)
Parameter containing:
tensor([-0.6285,  0.0233,  0.0242,  0.6759], requires_grad=True)


In [3]:
w2, w1 = torch.linalg.qr(l.weight, mode='reduced')
b = l.bias
b1 = torch.zeros(w1.shape[0])
b2 = b
print(b2.shape)
print(b1.shape)
print(f"l.shape={(l.weight.shape, l.bias.shape)}")

l1 = nn.Linear(w1.shape[1], w1.shape[0])
l1.weight.data = w1
l1.bias.data = b1
print(f"l1.shape={(l1.weight.shape, l1.bias.shape)}")

l2 = nn.Linear(w2.shape[1], w2.shape[0])
l2.weight.data = w2
l2.bias.data = b2
print(f"l2.shape={(l2.weight.shape, l2.bias.shape)}")

print(l2.weight @ l1.weight)
x = torch.randn(2,)
print(l(x))
print(l2(l1(x)))




torch.Size([4])
torch.Size([2])
l.shape=(torch.Size([4, 2]), torch.Size([4]))
l1.shape=(torch.Size([2, 2]), torch.Size([2]))
l2.shape=(torch.Size([4, 2]), torch.Size([4]))
tensor([[-0.4147, -0.0406],
        [-0.4577,  0.0653],
        [-0.1847, -0.2568],
        [ 0.6002, -0.2063]], grad_fn=<MmBackward0>)
tensor([-0.9493, -0.2121, -0.3761,  0.8543], grad_fn=<AddBackward0>)
tensor([-0.9493, -0.2121, -0.3761,  0.8543], grad_fn=<AddBackward0>)


In [4]:
class SplitType(Enum):
    ADD = 1
    MUL = 2

def get_random_split_type():
    return SplitType(random.randint(1, 2))

In [5]:
import random


class SplitType(Enum):
    ADD = 1
    MUL = 2

def get_random_split_type():
    return SplitType(random.randint(1, 2))

class Kernel(nn.Module):
    def __init__(self, weigth: torch.Tensor, bias: torch.Tensor, activation: nn.Module = nn.SELU(), device: str = "cpu"):
        super(Kernel, self).__init__()
        self.device = device
        self.kernel = nn.Linear(weigth.shape[1], weigth.shape[0])
        self.kernel.weight.data = weigth
        self.kernel.bias.data = bias
        self.activation = activation
        self.seq = nn.Sequential(self.kernel, self.activation)
        self.flatten = nn.Flatten()
        self.left = None
        self.right = None
        self.split_mode = None

    def forward(self, x):
        x = self.flatten(x)
        # assert self.split_mode is None and self.left is None and self.right is None, f"split_mode={self.split_mode} but left={self.left} and right={self.right}"
        if(self.is_leaf()):
            return self.seq(x)
        elif(self.split_mode == SplitType.ADD):
            return (self.left(x) + self.right(x)) * 0.5
        elif(self.split_mode == SplitType.MUL):
            return self.right(self.left(x))
        else:
            raise ValueError(f"Unknown split mode {self.split_mode}")
    
    def calc_grad_mean(self):
        '''Вычисляет среднее всех абсолютных величин градиентов параметров'''
        mean = torch.tensor(0.0)
        if(not self.is_leaf()): return mean
        params = self.model.parameters(recurse=False)
        with torch.no_grad():
            n = 0
            for p in params:
                if p.grad is not None:
                    n += 1
                    mean += p.grad.abs().mean()
            mean /= n
        return mean
    
   
    def is_leaf(self):
        return self.left is None or self.right is None

    def split_add(self):
        # Для kernel = (right(x) + left(x)) * 0.5
        assert self.is_leaf(), f"split_add must perform only for leaf, but there are {self.left} and {self.right}"
        # device = next(self.kernel.parameters()).device
        self.left = copy.deepcopy(self).to(self.device)
        self.right = copy.deepcopy(self).to(self.device)
        self.kernel = None
        self.split_mode = SplitType.ADD        

    def split_mul(self):
        assert self.is_leaf(), f"split_mul must perform only for leaf, but there are {self.left} and {self.right}"
        # Для kernel = right(left(x))
        # W*x+b = W2*(W1*x+b1)+b2 = W2*W1*x+W2*b1+b2
        # W2*W1 = W для поиска W2,W1 используем QR разложение
        # Примем b1 = 0, тогда b2 = b
        w = self.kernel.weight
        b = self.kernel.bias
        w2, w1 = torch.linalg.qr(w, mode='reduced')
        b1 = torch.zeros(w1.shape[0])
        b2 = b        
        self.left = Kernel(w1, b1, device=self.device).to(self.device)
        self.right = Kernel(w2, b2, device=self.device).to(self.device)
        self.kernel = None
        self.split_mode = SplitType.MUL
        
    
    def split(self):
        if(self.is_leaf()):
            self.split_add()
            self.left.split_mul()
            self.right.split_mul()
        else:
            self.left.split()
            self.right.split()
    
    def split_random_step(self):
        if(self.is_leaf()):
            split_type = get_random_split_type()
            if(split_type == SplitType.ADD):
                self.split_add()
            elif(split_type == SplitType.MUL):
                self.split_mul()
        else:
            if(random.random()<0.5):
                self.left.split_random_step()
            else:
                self.right.split_random_step()
            

in_features = 2
out_features = 4
# Генерируем случайные веса и смещения
w = torch.randn(out_features, in_features)
b = torch.randn(out_features)
# Создаем модель
model = Kernel(w, b, device="cpu")
print(model)
# Проверяем, что она работает
x = torch.randn(1, in_features)
y = model(x)

# Разделяем его
model.split_mul()
# Проверяем, что она работает
print(model(x))
print(y)


Kernel(
  (kernel): Linear(in_features=2, out_features=4, bias=True)
  (activation): SELU()
  (seq): Sequential(
    (0): Linear(in_features=2, out_features=4, bias=True)
    (1): SELU()
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
)
tensor([[ 0.0533,  2.1987, -0.5621,  1.1301]], grad_fn=<EluBackward0>)
tensor([[-0.1372,  2.0312, -0.5256,  1.0323]], grad_fn=<EluBackward0>)


In [6]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)
batch_size = 2**14

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([10000, 1, 28, 28])
Shape of y: torch.Size([10000]) torch.int64


In [7]:
# Get cpu, gpu or mps device for training.
# device = "cuda"
# (
#     "cuda"
#     if torch.cuda.is_available()
#     else "mps"
#     if torch.backends.mps.is_available()
#     else "cpu"
# )
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Using {device} device")

in_features = 28*28
out_features = 10
# w = torch.randn(out_features, in_features)
# b = torch.randn(out_features)
# # Создаем модель

# model = Kernel(w, b).to(device)
# print(model)

Using cuda:0 device


In [8]:
# Класс обертка над Kernel для возможности управления делением ядер
class SupervisorModel(nn.Module):
    def __init__(self, in_features: int, out_features: int, kernel_size: int = 8, device: str = "cpu"):
        super().__init__()
        self.device = device
        self.in_features = in_features
        self.out_features = out_features
        self.kernel_size = kernel_size
        self.flatten = nn.Flatten()
        w = torch.randn(self.kernel_size, self.kernel_size)
        b = torch.randn(self.kernel_size)
        self.kernel = Kernel(w, b, device=self.device)
        self.model = nn.Sequential(            
            nn.Linear(in_features, kernel_size),
            self.kernel,
            nn.Linear(kernel_size, out_features)
            )

    def forward(self, x):
        x = self.flatten(x)
        return self.model(x)

    def split_random(self):
        self.kernel.split_random_step()


In [9]:
model = SupervisorModel(in_features, out_features, 64, device).to(device)
print(model)

SupervisorModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (kernel): Kernel(
    (kernel): Linear(in_features=64, out_features=64, bias=True)
    (activation): SELU()
    (seq): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): SELU()
    )
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (model): Sequential(
    (0): Linear(in_features=784, out_features=64, bias=True)
    (1): Kernel(
      (kernel): Linear(in_features=64, out_features=64, bias=True)
      (activation): SELU()
      (seq): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): SELU()
      )
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (2): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 2 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

evolutions = 64
for e in range(evolutions):
    c = sum(p.numel() for p in model.parameters())
    print(f"Evolution {e+1}\n*********************************************")
    print(f"Params={c}")
    epochs = 8
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        test(test_dataloader, model, loss_fn)
    model.split_random()
    
print("Done!")

Evolution 1
*********************************************
Params=55050
Epoch 1
-------------------------------
loss: 2.793095  [16384/60000]
loss: 2.558595  [49152/60000]
Test Error: 
 Accuracy: 10.5%, Avg loss: 2.434660 

Epoch 2
-------------------------------
loss: 2.429502  [16384/60000]
loss: 2.342494  [49152/60000]
Test Error: 
 Accuracy: 14.7%, Avg loss: 2.279345 

Epoch 3
-------------------------------
loss: 2.274088  [16384/60000]
loss: 2.214160  [49152/60000]
Test Error: 
 Accuracy: 19.3%, Avg loss: 2.165634 

Epoch 4
-------------------------------
loss: 2.160770  [16384/60000]
loss: 2.109282  [49152/60000]
Test Error: 
 Accuracy: 24.7%, Avg loss: 2.068479 

Epoch 5
-------------------------------
loss: 2.063811  [16384/60000]
loss: 2.017225  [49152/60000]
Test Error: 
 Accuracy: 30.3%, Avg loss: 1.982132 

Epoch 6
-------------------------------
loss: 1.977349  [16384/60000]
loss: 1.934452  [49152/60000]
Test Error: 
 Accuracy: 35.7%, Avg loss: 1.904032 

Epoch 7
---------

In [12]:
from torchview import draw_graph

In [13]:
model_graph = draw_graph(
    model, 
    input_size=(1, 28*28),
    graph_name='test',
    graph_dir='LR',
    depth=8,
    hide_inner_tensors=True,
    hide_module_functions=True,
    save_graph=True,
    expand_nested=True,
    show_shapes=True,
    filename='test',
    directory='.'
)